In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [518 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,244 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,266 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,337 kB]
Hit

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="yyyy-mm-dd")


In [4]:
# Show the df
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')


In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query_1 = """
SELECT YEAR(date) AS YEAR_BUILT, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms == 4
GROUP BY YEAR_BUILT
ORDER BY YEAR_BUILT
"""
spark.sql(query_1).show()

+----------+---------+
|YEAR_BUILT|avg_price|
+----------+---------+
|      2019| 300263.7|
|      2020|298353.78|
|      2021|301819.44|
|      2022|296363.88|
+----------+---------+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query_2 = """
SELECT YEAR(date) AS YEAR_BUILT, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE (bedrooms == 3 AND bathrooms == 3)
GROUP BY YEAR_BUILT
ORDER BY YEAR_BUILT
"""
spark.sql(query_2).show()

+----------+---------+
|YEAR_BUILT|avg_price|
+----------+---------+
|      2019|287287.82|
|      2020|294204.16|
|      2021|294211.46|
|      2022|292725.69|
+----------+---------+



In [9]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query_3 = """
SELECT YEAR(date) AS YEAR_BUILT, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE (bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000)
GROUP BY YEAR_BUILT
ORDER BY YEAR_BUILT
"""
spark.sql(query_3).show()

+----------+---------+
|YEAR_BUILT|avg_price|
+----------+---------+
|      2019|289859.14|
|      2020|292289.09|
|      2021|296330.96|
|      2022|290242.99|
+----------+---------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING ROUND(AVG(price), 2) >= 350000
ORDER BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
|  70|695865.58|
+----+---------+
only showing top 20 rows

--- 0.8297855854034424 seconds ---


In [11]:
# 7. Cache the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [13]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING ROUND(AVG(price), 2) >= 350000
ORDER BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
|  70|695865.58|
+----+---------+
only showing top 20 rows

--- 0.8877096176147461 seconds ---


In [14]:
# uncached data: 0.6731657981872559 seconds
# cached data: 0.6522655487060547 seconds

In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.parquet('parquet_home_sales', mode='overwrite')

In [16]:
# 11. Read the parquet formatted data.
p_home_sales = spark.read.parquet('parquet_home_sales')

In [17]:
# 12. Create a temporary table for the parquet data.
p_home_sales.createOrReplaceTempView('home_sales')

In [18]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.
start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING ROUND(AVG(price), 2) >= 350000
ORDER BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
|  70|695865.58|
+----+---------+
only showing top 20 rows

--- 0.6840918064117432 seconds ---


In [19]:
# uncached data: 0.6644017696380615 seconds
# partitioned data: 0.5540437698364258 seconds

In [20]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [21]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False